# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False
import json
import warnings
warnings.filterwarnings("ignore")

import pymysql
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
f_path="D:/myfile/p站数据/当月数据"
#f_path="F:/下班前订单数据"
#f_path="F:/订单日报/临时导出"

In [4]:
import os
excel_names = []
excel_date = []
for excel_name in os.listdir(f_path):
    excel_names.append(excel_name)
    excel_date.append(excel_name.split('.')[0])
print(excel_names)
print(excel_date)


['0611-0624.xlsx']
['0611-0624']


In [5]:
# df_list=[]
# for excel_name in excel_names:
#      # 读取每个excel到df
#     excel_path = f"{f_path}/{excel_name}"
#     print(excel_path)
#     df_split = pd.read_excel(excel_path,skiprows=0)
#     print(df_split)
#     df_list.append(df_split)
#     print(df_list)

# df = pd.concat(df_list,ignore_index=True)

In [6]:
df = pd.read_excel(f'D:/myfile/p站数据/当月数据/{excel_date[-1]}.xlsx')
df["下单日期"]=df["下单时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month

In [7]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [8]:
sql_cjz = '''
select o.order_number,o.alipay_order_id,o.`status`,o.create_time from db_digua_business.t_order as o 
left join db_digua_business.t_order_details as od on o.id=od.order_id where o.channel = '2283' and o.alipay_order_id is not NULL 
'''
df_cjz = query(sql_cjz)

# 信息定义

In [9]:
df = df[df['商品属性'].notnull()]

In [10]:
df["拒绝理由"]=df["备注信息"].str.split("拒绝理由：").str[1]
df["拒绝理由"]=df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')
df["取消原因"]=df["备注信息"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["备注信息"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
#df["无法联系原因"]=df["备注信息"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["下单商品"].str.contains('99新')|df["下单商品"].str.contains('95新')|df["下单商品"].str.contains('准新'),"二手","全新")
df.loc[:,"租赁方案"]=np.where(df["商品属性"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["预授权占用自有资金"]>0,"部分免押","全免押")
df.loc[:,"是否使用优惠券"]=np.where((df["设置买断金额"]>df["总租金"])&(df["租赁方案"]=="租完即送"),"使用优惠券","未使用优惠券")
df.loc[:,"是否进行预授权"]=np.where(df["预授权占用自有资金"].isnull(),"未预授权","已预授权")

In [11]:
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["商品属性"]),axis=1)

In [12]:
def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["商品属性"]),axis=1)

In [13]:
df.loc[:,"进件"]=np.where((df["订单状态"]=="待支付")|(df["订单状态"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

进件
未进件    114808
进件      28435
Name: count, dtype: int64

# 渠道归属

In [14]:
df.loc[:,"来源渠道"]=df["来源渠道"].fillna("未知渠道")

In [15]:
def qudao_type(a,b):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "社群" in a:
        return "支付宝社群"
    else :
        return a

In [16]:
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["活动名称"]),axis=1)

# 数据处理

In [17]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["订单状态"].map(dict_status_code)

In [18]:
df.sort_values(by=["下单日期","状态编码"],inplace=True)

In [19]:
#删除重复单号
df.drop_duplicates(subset=["单号"],inplace=True)
#删除身份证空值行
df.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['活动名称']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["订单状态"],axis=0,inplace=True)
df.drop(df[df['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)

In [20]:
#删除重复订单
df.drop_duplicates(subset=["单号"],inplace=True)
df.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)

# 定义状态

In [21]:
def reject_type(a,b,c,d,e,f,g):
    if a is not np.nan and b=="未进件":
        return "前置拦截"
    elif g is np.nan and b=="未进件":
        return "进件前取消"
    elif a is not np.nan and b=="进件":
        return "机审强拒"
    elif c is not np.nan and"抖音非免押用户" in c:
        return "抖音非免押拒绝"
    elif a is np.nan and b=="进件" and c is not np.nan and  '已退款' in e:
        return "人审拒绝"
    elif a is np.nan and b=="进件" and d is not np.nan:
        return "客户取消"
    #elif a is np.nan and b=="进件" and f is not np.nan:
        #return "无法联系"
    elif g is not np.nan and b=="进件" and "蚂蚁数控风险等级" in g:
        return "出库前风控强拒"
    elif g is not np.nan and b=="进件" and "命中出库前风控流强拒" in g:
        return "出库前风控强拒"
    elif a is np.nan and b=="进件" and "待审核" in e:
        return "待审核"
    elif a is np.nan and b=="进件" and "待发货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "待收货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "租赁中" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    else :
        return e
    
    

In [22]:
df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["订单状态"],x["订单状态"],x["备注信息"]),axis=1)

In [23]:

fw_path = 'D:/myfile/p站数据/服务订单'
fw_excel_names = []
fw_excel_date = []
for excel_name in os.listdir(fw_path):
    fw_excel_names.append(excel_name)
    fw_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(fw_excel_date)


['服务订单0611-0624']


In [24]:
f_path_insure=f"D:/myfile/p站数据/服务订单/{fw_excel_date[-1]}.xlsx"
df_insure=pd.read_excel(f_path_insure,skiprows=0)
df_insure1=pd.merge(df,df_insure,left_on="单号", right_on="租赁订单号",how="left")
df=df_insure1

In [25]:
df_jj=df[df["进件"]=="进件"]
dff = df

In [26]:
import datetime

Today=datetime.date.today()
str(Today)
Now=str(datetime.datetime.now() )
Now

'2025-07-17 16:56:13.571814'

# 删除商家数据

In [27]:
df.drop(df[df['商家']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="优优2店"].index,inplace=True)
df.drop(df[df['商家']=="探路者自营账户"].index,inplace=True)
df.drop(df[df['商家']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['商家']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['商家']=="北京海鸟窝科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="澄心优租"].index,inplace=True)
df.drop(df[df['商品型号'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
# 趣智数码  单
df.drop(df[df['商家']=="趣智数码"].index,inplace=True)
df.drop(df[df['商家']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['商家']=="广州康基贸易有限公司"].index,inplace=True)
df.drop(df[df['商家']=="汇客好租"].index,inplace=True)
# df.drop(df[df['商家']=="线下小店"].index,inplace=True)
df.drop(df[df['商家']=="乙辉数码"].index,inplace=True)
df.drop(df[df['商家']=="南京聚格网络科技"].index,inplace=True)
df.drop(df[df['商家']=="呱子笔记本电脑"].index,inplace=True)

In [28]:
df_j=df[df["进件"]=="进件"]

# 以下为出库相关数据，要统计到出库相关的数据，必须运行到以下读取出库台账的代码

In [29]:
#读取导入出库台账数据
f_path_ck="D:/excel_data/台账/维客壹佰2025年台账.xlsx"
df_ck=pd.read_excel(f_path_ck,sheet_name="2025",skiprows=0)
dfck=pd.merge(df_ck,df,left_on="订单号", right_on="单号")
dfck.drop_duplicates(subset=["订单号"],inplace=True)


In [30]:
dffck=pd.merge(df_ck,dff,left_on="订单号", right_on="单号")
dffck.drop_duplicates(subset=["订单号"],inplace=True)

 # 检查出库订单中有没有客户取消的单，即订单状态为”已退款"的单，找出来删除后重新运行上一块导入出库台账的代码

In [31]:
dfck.drop(dfck[dfck["订单状态"]=="已退款"].index,inplace=True)
dffck.drop(dffck[dffck["订单状态"]=="已退款"].index,inplace=True)

In [32]:

try:
    dffck.drop(dffck[dffck["类目"]=="露营设备"].index,inplace=True)
except:
    dffck.drop(dffck[dffck["类型"]=="露营设备"].index,inplace=True)

# 芝麻租物各机型内存转化（瑞彤需求）

In [33]:
def product(df, model):
    df1=pd.crosstab(df[model],df["订单状态"],margins=True)
    df1=df1.rename(columns={"All":"去重订单数"})
    df2=pd.crosstab(df[model],df["进件"],margins=True)
    df1_2_merge=pd.merge(df1,df2["进件"],on=model,how='outer')
    df3=pd.crosstab(df[model],df["审核状态"],margins=True)
    if '客户取消' not in df3.columns :
        df3.loc[:, '客户取消'] = 0
    if '机审强拒' not in df3.columns:
        df3.loc[:, '机审强拒'] = 0
    df123_merge=pd.merge(df1_2_merge,df3[["前置拦截", '进件前取消',"客户取消","人审拒绝", '机审强拒']],on=model,how='outer')

    df123_merge["拦截率"]=df123_merge["前置拦截"]/df123_merge["去重订单数"]
    df123_merge["拦截率"]=df123_merge["拦截率"].apply(lambda x:format(x,".2%"))

    df123_merge["预授权通过率"]=df123_merge["进件"]/df123_merge["去重订单数"]
    df123_merge["预授权通过率"]=df123_merge["预授权通过率"].apply(lambda x:format(x,".2%"))

    df123_merge["机审通过件"]=df123_merge["进件"]-df123_merge["机审强拒"]

    df123_merge["风控通过件"]=df123_merge["进件"]-df123_merge["人审拒绝"]-df123_merge["机审强拒"]
    df123_merge["风控通过率"]=df123_merge["风控通过件"]/df123_merge["进件"]
    df123_merge["风控通过率"]=df123_merge["风控通过率"].apply(lambda x:format(x,".2%"))

    df123_merge["取消率"]=df123_merge["客户取消"]/df123_merge["进件"]
    df123_merge["取消率"]=df123_merge["取消率"].apply(lambda x:format(x,".2%"))

    df123_merge["人审拒绝率"]=df123_merge["人审拒绝"]/df123_merge["进件"]
    df123_merge["人审拒绝率"]=df123_merge["人审拒绝率"].apply(lambda x:format(x,".2%"))
    
    ck_list = ["待发货", "待收货", "租赁中","已完成"]
    df123_merge["出库"] = 0
    for ck in ck_list:
        if ck in df123_merge.columns:
            df123_merge["出库"] += df123_merge[ck]
    
    df123_merge["进件出库率"]=df123_merge["出库"]/df123_merge["进件"]
    df123_merge["进件出库率"]=df123_merge["进件出库率"].apply(lambda x:format(x,".2%"))

    df123_merge["订单出库率"]=df123_merge["出库"]/df123_merge["去重订单数"]
    df123_merge["订单出库率"]=df123_merge["订单出库率"].apply(lambda x:format(x,".2%"))

    df_bfmy=df123_merge[["去重订单数","前置拦截","拦截率", '进件前取消', "进件","预授权通过率","机审通过件",
                        "人审拒绝","风控通过件","风控通过率","客户取消","出库","进件出库率","取消率","人审拒绝率","订单出库率"]]
    
    return df_bfmy

## 总体

In [34]:
df["型号内存"]=df["商品型号"]+"_"+df["内存"]
dfcs=df[df["商品类型"]=="全新"]
# dfcs=df[(df["商品类型"]=="二手")&(df['商品型号'].str.contains(r'iPhone ?16')==True)]


dfcs_group = product(dfcs, '型号内存')
dfcs_group = dfcs_group.sort_values(by=["去重订单数"],ascending=False)
dfcs_group.to_excel(f"D:/芝麻租物各机型转化内存/{excel_date[-1]}总体各机型内存转化数据（全新）.xlsx")

## 芝麻租物

In [35]:
df_zm=dfcs[dfcs["归属渠道"]=="芝麻租物"]
df_zm_group = product(df_zm, '型号内存')
# df_zm = df_zm[df_zm.商品id.isin([6713,6712,6711,6710, 7681, 7684])]
# df_zm = df_cjz.merge(df_zm, left_on='order_number', right_on='单号',  how='inner')
# df_zm_group = product(df_zm, '商品id')

df_zm_group.sort_values(by=["去重订单数"],ascending=False)
df_zm_group.to_excel(f"D:/芝麻租物各机型转化内存/{excel_date[-1]}芝麻租物各机型内存转化数据（全新）.xlsx")

In [36]:
# df_zm[df_zm.进件=='进件'][['单号', '商品型号']].to_excel('F:/需求/总需求/芝麻进件商品型号.xlsx')

## 搜索渠道

In [37]:
df_ss=dfcs[dfcs["归属渠道"]=="搜索渠道"]
df_ss_group = product(df_ss, '型号内存')
# df_ss_group = df_ss_group.sort_values(by=["去重订单数"],ascending=False).to_excel(f"F:/芝麻租物各机型转化内存/{excel_date[-1]}搜索渠道各机型内存转化数据（全新）.xlsx")

# 周数据

## 总体

In [38]:
week_dfcs_group = product(dfcs, '下单日期')[:-1][['去重订单数', '前置拦截', '进件', '机审通过件', '人审拒绝', '风控通过件', '客户取消', '出库']]
week_dfcs_group.rename(columns={'进件': '进件数'}, inplace=True)
week_dfcs_group = week_dfcs_group.reset_index()
week_dfcs_group['下单日期'] = pd.to_datetime(week_dfcs_group['下单日期'])
week_dfcs_group.set_index('下单日期', inplace=True)
week_dfcs_group_all = week_models.week_data_group_all(week_dfcs_group, 2)[['week_group','去重订单数', '前置拦截', '拦截率','进件数','预授权通过率','机审通过件','人审拒绝', '风控通过件', '风控通过件比例', '客户取消', '出库', '进件出库率', '客户取消比例', '人审拒绝比例', '订单出库率']]

In [39]:
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/总体各机型内存转化数据（全新）.xlsx_{excel_date[-1]}', engine='openpyxl') as writer:
#     week_dfcs_group_all.to_excel(writer, sheet_name='总体周数据')
#     dfcs_group.to_excel(writer, sheet_name='总体数据明细')

## 搜索渠道

In [40]:
week_ss_group = product(df_ss, '下单日期')[:-1][['去重订单数', '前置拦截', '进件', '机审通过件', '人审拒绝', '风控通过件', '客户取消', '出库']]
week_ss_group.rename(columns={'进件': '进件数'}, inplace=True)
week_ss_group = week_ss_group.reset_index()
week_ss_group['下单日期'] = pd.to_datetime(week_ss_group['下单日期'])
week_ss_group.set_index('下单日期', inplace=True)
week_ss_group = week_models.week_data_group_all(week_ss_group, 2)[['week_group','去重订单数', '前置拦截', '拦截率','进件数','预授权通过率','机审通过件','人审拒绝', '风控通过件', '风控通过件比例', '客户取消', '出库', '进件出库率', '客户取消比例', '人审拒绝比例', '订单出库率']]

In [41]:
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/搜索渠道各机型内存转化数据（全新）_{excel_date[-1]}.xlsx', engine='openpyxl') as writer:
#     week_ss_group.to_excel(writer, sheet_name='搜索渠道周数据', index=False)
#     df_ss_group.to_excel(writer, sheet_name='搜索渠道数据明细')

## 芝麻租物

In [42]:
week_zm_group = product(df_zm, '下单日期')[:-1][['去重订单数', '前置拦截', '进件', '机审通过件', '人审拒绝', '风控通过件', '客户取消', '出库']]
week_zm_group.rename(columns={'进件': '进件数'}, inplace=True)
week_zm_group = week_zm_group.reset_index()
week_zm_group['下单日期'] = pd.to_datetime(week_zm_group['下单日期'])
week_zm_group.set_index('下单日期', inplace=True)
week_zm_group = week_models.week_data_group_all(week_zm_group, 2)[['week_group', '去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审通过件', '人审拒绝', '风控通过件', '风控通过件比例', '客户取消', '出库', '进件出库率', '客户取消比例', '人审拒绝比例', '订单出库率']]

In [44]:
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/芝麻租物各机型内存转化数据（全新）_{excel_date[-1]}.xlsx', engine='openpyxl') as writer:
#     week_zm_group.to_excel(writer, sheet_name='芝麻租物周数据', index=False)
#     df_zm_group.to_excel(writer, sheet_name='芝麻租物数据明细')
week_zm_group

,week_group,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,人审拒绝,风控通过件,风控通过件比例,客户取消,出库,进件出库率,客户取消比例,人审拒绝比例,订单出库率
0,20250611-20250617,19485,361,1.85%,4287,22.00%,2913,1109,1804,42.08%,276,753,17.56%,6.44%,25.87%,3.86%
1,20250618-20250624,20951,426,2.03%,4615,22.03%,3015,1081,1934,41.91%,321,801,17.36%,6.96%,23.42%,3.82%
